<a href="https://colab.research.google.com/github/Colsai/Covid-Visualizations-And-Trends/blob/main/covid_dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# US Covid Dataset (NYT Expanded)
<blockquote> Scott Hirabayashi | (Sourced from NYT's Github) | 2020  </blockquote>

- Adding Cases Increase
- Deaths Increase 
- Day of Week
- Percent Change
- etc.

to dataset


## Import Packages
We will use datetime, pandas, numpy, and statistics 

In [32]:
#Datetime To Convert Dates
from datetime import datetime

#Import Packages for Data
import pandas as pd
import numpy as np
import statistics

## Import the data from NYT github
NYT's github has national/state/local data.  
We'll use the US data for this manipulation.

In [33]:
#Cases by day from beginning (US)

#USA Data will be the dataframe for national data
site = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv'
us_df = pd.read_csv(site)

Let's look at the head of the data first...

In [34]:
#Let's look at the head of us df first
us_df.head(5)

,date,cases,deaths
0,2020-01-21,1,0
1,2020-01-22,1,0
2,2020-01-23,1,0
3,2020-01-24,2,0
4,2020-01-25,3,0


Only Cases and Deaths here.  
Let's look to add some more features. 

## Let's look at US DF First
Looking at the data, we'll have to change the dates to Datetime

In [35]:
#One issue here is that the date data is currently string. We need to make it datetime.
type(us_df['date'][0])

str

In [36]:
#Change the items into datetime (rather than string)
us_df['date'] = pd.to_datetime(us_df['date'])

In [37]:
#Did it work?
type(us_df['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

So, it worked. Now we can graph this normally instead of having more issues. 

## Add days after...
One of the issues with datetime is that some of the data can't be plotted with datetime (SNS regplot, for example).

So that we can look at it with seaborn, etc, we'll add a new column.

In [38]:
#This function just appends to each data frame the number of days, so we don't have issues later with time data
days_after = []
for i in range(1,len(us_df['date'].to_list()) + 1):
    days_after.append(i)

us_df.insert(1,"days_since_start", days_after)

Let's also create a function for total change.

In [39]:
#This is a simple function that returns items based on their change over time. 
def total_change(input_list, rounded_val=2):
    total_list = []
    day_change = 0
    
    for day in input_list:
        try:
              day_change = day - day_before
        except:
              day_before = 0

        total_list.append(day_change)
        day_before = day
    return total_list

And check to see what items and types of data we have here so far...

In [40]:
#What are the data types in here?
us_df.dtypes

date                datetime64[ns]
days_since_start             int64
cases                        int64
deaths                       int64
dtype: object

## Add day of the week
One helpful thing might be to see what day of the week it is.  
This way, we can get a feel here for the days that might show more cases in a given week.

In [41]:
#Function to return and add day of week
day_num = ["Monday", "Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

def get_day(list_dates):
    day_of_week = []
    
    for days in list_dates:
        weekday_num = days.weekday()
        day_name = day_num[weekday_num]
        day_of_week.append(day_name) 
    return day_of_week

Let's check the head of the data again

In [42]:
#Head of the data
us_df.head()

,date,days_since_start,cases,deaths
0,2020-01-21,1,1,0
1,2020-01-22,2,1,0
2,2020-01-23,3,1,0
3,2020-01-24,4,2,0
4,2020-01-25,5,3,0


In [43]:
#What are the data types?
us_df['deaths'].dtypes

dtype('int64')

In [44]:
#This is a simple function that returns items based on their change over time. 
def percent_change(input_list, rounded_val=2):
    percent_list = []

    for this_year in input_list:
        try:
              pct_increase = ((this_year - last_year) / last_year) * 100
        except:
              pct_increase = 0

        percent_list.append(round(pct_increase,2))
        last_year = this_year

    percent_list = [round(i, rounded_val) for i in percent_list] #This function combines rounding inside of it, since percents are easier to use

    return percent_list

In [45]:
#Run the Percent Change Functions
cases_new = total_change(us_df['cases'].to_list())
deaths_new = total_change(us_df['deaths'].to_list())

In [46]:
us_df.insert(3, "cases_increase", cases_new)
us_df.insert(5, "deaths_increase", deaths_new)

In [47]:
#Run the Percent Change Functions
cases_pct = percent_change(us_df['cases'].to_list())
deaths_pct = percent_change(us_df['deaths'].to_list())

In [48]:
#Insert the Two Into the USA df
us_df.insert(4, "cases_pct_inc", cases_pct)
us_df.insert(7, "deaths_pct_inc", deaths_pct)

In [49]:
us_df['deaths_increase'].describe().astype(int)

count     337
mean      958
std       772
min         0
25%       395
50%       891
75%      1327
max      3611
Name: deaths_increase, dtype: int64

Day of Week

In [50]:
dates = us_df['date'].to_list()

In [51]:
us_df.insert(1, "day_of_week",get_day(dates))

In [52]:
us_df.head()

,date,day_of_week,days_since_start,cases,cases_increase,cases_pct_inc,deaths,deaths_increase,deaths_pct_inc
0,2020-01-21,Tuesday,1,1,0,0.0,0,0,0.0
1,2020-01-22,Wednesday,2,1,0,0.0,0,0,0.0
2,2020-01-23,Thursday,3,1,0,0.0,0,0,0.0
3,2020-01-24,Friday,4,2,1,100.0,0,0,0.0
4,2020-01-25,Saturday,5,3,1,50.0,0,0,0.0


Aggregate them by week

In [53]:
week_agg = us_df.groupby("day_of_week",  sort=False).agg("mean")

In [54]:
week_agg = week_agg.reset_index()

## Cases and Deaths Inclusion
Adding Rolling Averages so that we can graph them later.

In [55]:
#1 week, 2 week, 6 week
for i in range(14,1,-7):
    us_df.insert(4, f'{i}_day_cases', us_df['cases_increase'].rolling(i).mean())

for i in range(14,1,-7):
    us_df.insert(9, f'{i}_day_deaths', us_df['deaths_increase'].rolling(i).mean())

us_df.insert(6, '42_day_cases', us_df['cases_increase'].rolling(42).mean())
us_df.insert(12, f'42_day_deaths', us_df['deaths_increase'].rolling(42).mean())

In [56]:
us_df.tail(15)

,date,day_of_week,days_since_start,cases,7_day_cases,14_day_cases,42_day_cases,cases_increase,cases_pct_inc,deaths,7_day_deaths,14_day_deaths,42_day_deaths,deaths_increase,deaths_pct_inc
322,2020-12-08,Tuesday,323,15251704,207469.857143,184358.571429,152369.214286,219905,1.46,286667,2290.571429,1915.928571,1428.238095,2821,0.99
323,2020-12-09,Wednesday,324,15471382,210111.571429,187154.428571,155649.071429,219678,1.44,289824,2329.428571,1976.214286,1479.214286,3157,1.10
324,2020-12-10,Thursday,325,15696634,211153.428571,195879.285714,158850.452381,225252,1.46,292761,2340.857143,2101.785714,1525.238095,2937,1.01
325,2020-12-11,Friday,326,15977148,218201.000000,201240.142857,163152.833333,280514,1.79,295712,2385.714286,2211.714286,1572.380952,2951,1.01
326,2020-12-12,Saturday,327,16184592,218442.000000,205254.357143,166086.380952,207444,1.30,297971,2395.571429,2287.928571,1606.214286,2259,0.76
327,2020-12-13,Sunday,328,16368840,220169.000000,208676.500000,168706.000000,184248,1.14,299328,2430.714286,2326.428571,1628.357143,1357,0.46
328,2020-12-14,Monday,329,16569913,219730.571429,211056.000000,171261.571429,201073,1.23,301006,2451.428571,2355.928571,1655.452381,1678,0.56
329,2020-12-15,Tuesday,330,16772913,217315.571429,212392.714286,173889.428571,203000,1.23,304031,2480.571429,2385.571429,1700.571429,3025,1.00
330,2020-12-16,Wednesday,331,17017946,220937.714286,215524.642857,177154.904762,245033,1.46,307642,2545.428571,2437.428571,1748.071429,3611,1.19
331,2020-12-17,Thursday,332,17256135,222785.857143,216969.642857,179932.238095,238189,1.40,310935,2596.285714,2468.571429,1800.095238,3293,1.07


## Add the week number
This lets us know which week we are on in the dataset

In [57]:
num_week = []
week = 0
for i in range(0,len(us_df)):
    if i%7 == 0:
        week+=1
    num_week.append(week)

In [58]:
us_df.insert(2, "num_week", num_week)

In [59]:
us_df.head(8)

,date,day_of_week,num_week,days_since_start,cases,7_day_cases,14_day_cases,42_day_cases,cases_increase,cases_pct_inc,deaths,7_day_deaths,14_day_deaths,42_day_deaths,deaths_increase,deaths_pct_inc
0,2020-01-21,Tuesday,1,1,1,NaN,NaN,NaN,0,0.00,0,NaN,NaN,NaN,0,0.0
1,2020-01-22,Wednesday,1,2,1,NaN,NaN,NaN,0,0.00,0,NaN,NaN,NaN,0,0.0
2,2020-01-23,Thursday,1,3,1,NaN,NaN,NaN,0,0.00,0,NaN,NaN,NaN,0,0.0
3,2020-01-24,Friday,1,4,2,NaN,NaN,NaN,1,100.00,0,NaN,NaN,NaN,0,0.0
4,2020-01-25,Saturday,1,5,3,NaN,NaN,NaN,1,50.00,0,NaN,NaN,NaN,0,0.0
5,2020-01-26,Sunday,1,6,5,NaN,NaN,NaN,2,66.67,0,NaN,NaN,NaN,0,0.0
6,2020-01-27,Monday,1,7,5,0.571429,NaN,NaN,0,0.00,0,0.0,NaN,NaN,0,0.0
7,2020-01-28,Tuesday,2,8,5,0.571429,NaN,NaN,0,0.00,0,0.0,NaN,NaN,0,0.0


In [63]:
us_df.groupby("num_week", as_index=False).agg("mean")

,num_week,days_since_start,cases,7_day_cases,14_day_cases,42_day_cases,cases_increase,cases_pct_inc,deaths,7_day_deaths,14_day_deaths,42_day_deaths,deaths_increase,deaths_pct_inc
0,1,4.0,2.571429e+00,0.571429,NaN,NaN,0.571429,30.952857,0.000000,0.000000,NaN,NaN,0.000000,0.000000
1,2,11.0,7.571429e+00,0.714286,0.714286,NaN,0.857143,12.637143,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
2,3,18.0,1.200000e+01,0.632653,0.673469,NaN,0.285714,2.488571,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
3,4,25.0,1.600000e+01,0.571429,0.602041,NaN,1.714286,11.642857,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
4,5,32.0,3.000000e+01,2.000000,1.285714,NaN,2.571429,8.920000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000
5,6,39.0,7.028571e+01,5.755102,3.877551,2.452381,8.714286,13.984286,1.428571,0.204082,0.102041,0.142857,0.857143,42.857143
6,7,46.0,3.640000e+02,41.959184,23.857143,8.605442,92.000000,32.738571,16.571429,2.163265,1.183673,0.394558,2.857143,24.615714
7,8,53.0,2.454000e+03,298.571429,170.265306,58.248299,537.000000,29.327143,54.285714,5.387755,3.775510,1.292517,9.285714,19.747143
8,9,60.0,2.082386e+04,2624.265306,1461.418367,495.520408,5571.142857,38.391429,309.285714,36.428571,20.908163,7.363946,69.714286,30.307143
9,10,67.0,1.059124e+05,12155.510204,7389.887755,2521.343537,17207.142857,20.947143,1906.857143,228.224490,132.326531,45.401361,398.428571,28.722857


Save the dataset

In [61]:
#Save the new us csv as a file
us_df.to_csv("expanded_covid_data.csv")